<a href="https://colab.research.google.com/github/ShintaG3/Evangeriot/blob/main/%E5%88%9D%E5%8F%B7%E6%A9%9FV1_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install ccxt
!pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.7/288.7 kB 30.3 MB/s eta 0:00:00


In [7]:
import requests
from datetime import datetime
from pprint import pprint
import time, calendar
from pytz import timezone
import numpy as np
import pandas as pd
import ccxt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from IPython.display import display

In [8]:
# ------------------------------
# 000-F初号機V1.0（本番用）
# ------------------------------
# 010BITMEXインスタンス
bitmex = ccxt.bitflyer({
    "apiKey":'Tm5TRcQKitGeCB44JsiTtC',                         #BitMEXアカウントで取得したAPIキーを入力。本番用のアカウントのAPIとテストネットのアカウントのAPIは別であることに注意
    "secret":'0+wgPF0AdenOCk73yi1PR/qzbYkeRZ3MTVxFvAI9vdI=',
    "timeout": 10000 #タイムアウトms
})
# bitmex.urls['api'] = bitmex.urls['test'] #本番用のアカウントで運用する場合はこの行をコメントアウトする
currency = 'BTC/JPY'

In [ ]:
# bitmex.publicGetGetboardstate() # get board state
bitmex.publicGetGetboard().keys()
bitmex.publicGetGetboard()['bids']

In [10]:
# bitmex.load_markets() # get available currency pairs
# bitmex.fetch_ticker(currency) # get ticker info
# bitmex.fetchOpenOrders(currency) # get open order >
# bitmex.privateGetGetchildorders(params={'child_order_state':'ACTIVE','product_code':'FX_BTC_JPY'}) # get open order
# bitmex.fetch_balance() # get account info
# curl https://api.bitflyer.com/v1/getexecutions?product_code=BTC_USD
# bitmex.milliseconds()
# bitmex.create_order() -> createOrder
# bitmex.describe()
# bitmex.private_get_position() # not avail ->
# bitmex.private_get_getpositions(params={'product_code':'FX_BTC_JPY'})
# bitmex.cancel_order(
#     id='JRF20231114-061121-063169', # child_order_acceptance_id
#     symbol=currency,
#     params = { "product_code" : "FX_BTC_JPY" }
# )
bitmex.privatePostCancelallchildorders(
    params = { "product_code" : "FX_BTC_JPY" }
)
# bitmex.private_delete_order_all() # not avail
# bitmex.fetch_ohlcv(symbol, timeframe, since, qty) # deprecated

''

In [11]:
bitmex.privatePostCancelallchildorders(
    params = { "product_code" : "FX_BTC_JPY" }
)

''

In [44]:
# side='buy'
# lot=0.01
# price=5776650.0
# order = bitmex.create_order(
# 				symbol = currency,
# 				type='limit', # 指値
# 				side= side,
# 				amount= lot,
# 				price=price,
#         params = { "product_code" : "FX_BTC_JPY" }
# 			)

In [38]:
# 020設定
# バックテスト用設定-----------------------------
backtest = False                # バックテスト実行の設定　True or False
collateral = 0.1               # バックテスト時の資産 単位：XBT
backtest_print_log = "OFF"     # バックテスト時のログ表示設定 "ON" or "OFF"
limit = 1000                # バックテストでテストするデータ数
#メインの設定------------------------------------
wait = 5                       # ループの待機時間　単位：秒（3秒以上）
timeframe = "1m"               # 時間足を指定。'1m' '5m' '15m' '30m''1h' '2h' '3h' '4h''6h' '12h' '1d'から選択
trade_risk = 0.03              # 1トレードあたり口座の何％まで損失を許容するか(0.01 = 1%)
check_order_time = 60          # 未約定の注文をキャンセルするまでの待機時間 t = 10sec （例：360は3600秒待機）
error_recovery_time = 60       # エラー発生時のスリープ時間（秒）
# ドンチアンブレイクの設定--------------------------
don_buy_term = 10              # 最高値（上値）ブレイクアウト期間
don_sell_term = 10             # 最安値（下値）ブレイクアウト期間
judge_price={
  "buy" : "high",              # ブレイク判断　高値（high)か終値（close）を使用
  "sell": "low",               # ブレイク判断　安値 (low)か終値（close）を使用
}
sma_filter = "OFF"              # 移動平均線のフィルターの適用 "ON" or "OFF"
# ポジション追加の設定----------------------------
entry_times = 2                # 追加ポジションを持つ回数
entry_range = 1                # 何ATR幅ごとに追加ポジションを持つか
# 損切り設定--------------------------------------
atr_term = 5                   # ATRの期間
atr_range = 2                  # 損切りに使用するATRの幅
# トレイリングストップ設定------------------------
trailing_config = "OFF"         # ONで有効 OFFで無効
stop_AF = 0.02                 # 加速係数
stop_AF_add = 0.02             # 加速係数を増やす度合
stop_AF_max = 0.3              # 加速係数の上限
# トレンドフィルター用----------------------------
last_data = pd.DataFrame()
alfa = 1                       #価格データの時間足（timeframe)と異なる時間足のインジケーターを使用する場合。例：価格データ15分足、インジケーター45分足の場合は alfa = 3
sma_term = 200                 #移動平均期間
sma_offset = 0                 #移動平均線を移動する期間（プラス：右へ移動 マイナス：左へ移動）
# LINE通知設定------------------------------------
line_config = "OFF"             # LINE通知をするかどうかの設定 "ON" or "OFF"
line_token = "LINEトークンを入力"     # LINE通知を使用する場合はAPIキーを入力
#定数(変更しない)----------------------------------
ALLOWED_TIMEFRAME = {
		"1m": ["1m", 1,    1, 1],  "3m": ["1m",  3,   3, 1],
		"5m": ["5m", 1,    5, 5], "15m": ["5m",  3,  15, 5], "30m": ["5m", 6, 30, 5],
		"1h": ["1h", 1,   60, 60],  "2h": ["1h",  2, 120, 60],
		"3h": ["1h", 3,  180, 60],  "4h": ["1h",  4, 240, 60],
		"6h": ["1h", 6,  360, 60], "12h": ["1h", 12, 720, 60],
		"1d": ["1d", 1, 1440, 1440],
}

In [13]:
# 030通知＆ファイル出力
# 031価格通知
def print_price(data):
	if backtest == False or backtest_print_log == "ON":
		print_log(f"時間：{data['datetime']} 高値：{data['high']} 安値：{data['low']} 終値：{str(data['close'])} USD 移動平均: {data['sma']}")


# 032コマンドライン&LINE通知&ファイル出力
def print_log(text):
	if backtest == False or backtest_print_log == "ON":
		# LINE通知する場合
		if line_config == "ON":
			url = "https://notify-api.line.me/api/notify"
			data = {"message" : str(text)}
			headers = {"Authorization": "Bearer " + line_token}
			try:
				requests.post(url, data=data, headers=headers)
			except requests.exceptions.RequestException as e:
				print(str(e))
		# コマンドラインへの表示
		print(text)

In [14]:
bitmex.privateGetGetchildorders(params={'child_order_state':'ACTIVE','product_code':'FX_BTC_JPY'})

[]

In [15]:
# ------------------------------
# 100約定確認
# ------------------------------
def check_open_order():
  t = 0
  while True:
    try:
      open_order = bitmex.privateGetGetchildorders(params={'child_order_state':'ACTIVE','product_code':'FX_BTC_JPY'})
      if open_order:
        flag["order"]["exist"] = True
        if t and t % check_order_time == 0: # 設定時間経過したか確認
          print_log(f"未約定の状態で{check_order_time*10}秒経過したため、すべての注文をキャンセルします。")
          bitmex.privatePostCancelallchildorders(
              params = { "product_code" : "FX_BTC_JPY" }
          )
          flag["order"]["exist"] = False
        else:
          print_log(f"{len(open_order)}つの未約定の注文があります。")
          _ = [print_log(f"{i+1}: サイド:{v['side'].upper()},ロット:{v['amount']},価格:{v['price']}USD\n") for i,v in enumerate(open_order)]
          print_log("注文がすべて約定するのを待っています")
          while True:
            open_order = bitmex.privateGetGetchildorders(params={'child_order_state':'ACTIVE','product_code':'FX_BTC_JPY'})
            if open_order:
              time.sleep(10)
              t += 1 #10秒ごとにtを加算
              if t and t % 3 == 0: #30秒ごとにブレイクしてログを表示
                break
            else:
              break
      else:
        if flag["order"]["exist"]:
          print_log("すべての注文が約定されました。")
        flag["order"]["exist"] = False
        break
    except ccxt.BaseError as e:
      print_log("約定確認のAPIで問題発生 : " + str(e))
      print_log(f"{error_recovery_time}秒待機してやり直します")
      time.sleep(error_recovery_time)

In [28]:
# ------------------------------
# 200価格データ取得
# ------------------------------
def get_price(timeframe="1m", symbol="BTC/USD", limit=500):
  while True:
    try:
      if timeframe not in ALLOWED_TIMEFRAME:
          return None
      timeframe_params = ALLOWED_TIMEFRAME[timeframe]
      need_limit = limit * timeframe_params[1]
      # APIリクエストでOHLCVデータ取得
      df_ohlcv = get_ohlcv_data(timeframe=timeframe_params[0], limit=need_limit)
      # 欠損値を補正するためにすべてのデータにおいてリサンプリングを適用する
      minutes = ALLOWED_TIMEFRAME[timeframe][2]
      offset = str(minutes) + "T"
      if 60 <= minutes < 1440:
          offset = str(minutes / 60) + "H"
      elif 1440 <= minutes:
          offset = str(minutes / 1440) + "D"
      df_ohlcv = df_ohlcv.resample(offset).agg({
                      "timestamp": "last",
                      "open":      "first",
                      "high":      "max",
                      "low":       "min",
                      "close":     "last",
                      "volume":    "sum",
                  }).interpolate() # interpolateメソッドで欠損値を前後の値から線形的に補正
      # index解除
      df_ohlcv.reset_index(inplace=True)
      # timestampをサンプリング時間に書き換え
      df_ohlcv["timestamp"] = pd.Series([int(dt.timestamp()) * 1000 for dt in df_ohlcv["datetime"]])
      # datetimeを日本時間に変換
      df_ohlcv['datetime'] = pd.Series([dt.astimezone(timezone('Asia/Tokyo')) for dt in df_ohlcv['datetime']])
      # マージした結果、余分に取得している場合、古い足から除去
      if len(df_ohlcv) > limit:
          df_ohlcv = df_ohlcv.iloc[len(df_ohlcv)-limit:]
      return df_ohlcv
    except ccxt.BaseError as e:
      print_log("価格データ取得API通信でエラー発生 : " + str(e))
      print_log(f"{error_recovery_time}秒待機してやり直します")
      time.sleep(error_recovery_time)


# 210フェッチ＆マージ
def __get_ohlcv_paged(symbol="BTC/USD", timeframe="1m", limit=1000):
  ohlcv_list = []
  restriction = 1000
  to_time = bitmex.milliseconds()
  since = to_time - ALLOWED_TIMEFRAME[timeframe][2] * 60 * 1000* limit
  qty = min(limit,restriction)
  while qty > 0 and len(ohlcv_list) < limit:
      fetch = bitmex.fetch_ohlcv(symbol, timeframe, since, qty) # depricated
      if len(fetch):
          since = fetch[-1][0] + (ALLOWED_TIMEFRAME[timeframe][3] * 60 * 1000)
          qty = min((limit - qty),restriction)
          ohlcv_list += fetch
      else:
          break
      time.sleep(1)
  df_ohlcv = pd.DataFrame(ohlcv_list, columns=["timestamp", "open", "high", "low", "close", "volume"])
  df_ohlcv["datetime"] = pd.to_datetime(df_ohlcv["timestamp"], unit="ms")
  df_ohlcv = df_ohlcv.set_index("datetime")
  df_ohlcv.index = df_ohlcv.index.tz_localize("UTC")
  return df_ohlcv

In [17]:
# 210フェッチ＆マージ
from datetime import datetime, timedelta
from time import sleep

import dateutil.parser
import pandas as pd
import requests

# Constants
TICK_RES = 60  # OHLCV data resolution (e.g., 60 for 1-minute intervals)
IDX_DATE = 0
IDX_OPEN = 1
IDX_HIGH = 2
IDX_LOW = 3
IDX_CLOSE = 4
IDX_VOLUME = 5

class RealtimeOHLCV:
    def __init__(self):
        self.ohlcvs = []
        self.current_ohlcv = [0, 0.0, 0.0, 0.0, 0.0, 0.0]

    def process_execution_data(self, executions, timeframe):
        for execution in executions:
            exec_date = self.get_exec_datetime(execution)
            self.create_ohlcv(exec_date, execution, timeframe)

    def create_ohlcv(self, exec_date, execution, timeframe):
        if self.is_next_tick(exec_date, timeframe):
            if self.current_ohlcv[IDX_DATE] != 0:
                self.ohlcvs.append(self.current_ohlcv.copy())
            self.init_ohlcv(exec_date, execution)
        else:
            self.update_ohlcv(exec_date, execution)

    def get_exec_datetime(self, execution):
        exec_date = execution["exec_date"].replace("T", " ")[:-1]
        return dateutil.parser.parse(exec_date) + timedelta(hours=9)

    def get_tick_datetime(self, dt):
        tick_ts = int(dt.timestamp() / TICK_RES) * TICK_RES
        return datetime.fromtimestamp(tick_ts)

    def is_next_tick(self, exec_date, timeframe):
        current_tick = self.get_tick_datetime(
            datetime.fromtimestamp(self.current_ohlcv[IDX_DATE])
        )
        new_tick = self.get_tick_datetime(exec_date)
        return current_tick != new_tick

    def init_ohlcv(self, exec_date, execution):
        price = float(execution["price"])
        vol = float(execution["size"])
        self.current_ohlcv[IDX_DATE] = self.get_tick_datetime(exec_date).timestamp()
        self.current_ohlcv[IDX_OPEN] = price
        self.current_ohlcv[IDX_HIGH] = price
        self.current_ohlcv[IDX_LOW] = price
        self.current_ohlcv[IDX_CLOSE] = price
        self.current_ohlcv[IDX_VOLUME] = vol

    def update_ohlcv(self, exec_date, execution):
        price = float(execution["price"])
        vol = float(execution["size"])
        if (
            self.current_ohlcv[IDX_DATE]
            == self.get_tick_datetime(exec_date).timestamp()
        ):
            self.current_ohlcv[IDX_HIGH] = max(self.current_ohlcv[IDX_HIGH], price)
            self.current_ohlcv[IDX_LOW] = min(self.current_ohlcv[IDX_LOW], price)
            self.current_ohlcv[IDX_CLOSE] = price
            self.current_ohlcv[IDX_VOLUME] += vol


def get_execution_data(timeframe, limit, params, rtOHLCV):
    endpoint = "https://api.bitflyer.com/v1/getexecutions"
    while len(rtOHLCV.ohlcvs) < limit:
        response = requests.get(endpoint, params=params)
        if response.status_code == 200:
            executions = response.json()
            if not executions:
                print("No more data available.")
                break

            rtOHLCV.process_execution_data(executions, timeframe)
            params["before"] = executions[-1]["id"]
            print(executions[-1]["exec_date"])
        else:
            if (
                "Execution history is limited to the most recent 31 days"
                in response.text
            ):
                print("Reached the 31-day limit.")
                break
            else:
                print("Failed to fetch data: Status code", response.status_code)

        sleep(0.61)


def get_ohlcv_data(timeframe, limit):
    rtOHLCV = RealtimeOHLCV()
    params = {}

    get_execution_data(timeframe, limit, params, rtOHLCV)

    df_ohlcv = pd.DataFrame(
        rtOHLCV.ohlcvs, columns=["timestamp", "open", "high", "low", "close", "volume"]
    )
    df_ohlcv["datetime"] = pd.to_datetime(df_ohlcv["timestamp"], unit="s")
    df_ohlcv = df_ohlcv.set_index("datetime")
    df_ohlcv.index = df_ohlcv.index.tz_localize("UTC")

    return df_ohlcv

In [18]:
# ------------------------------
# 300インジケーター作成
# ------------------------------
# 310単純移動平均
def sma(df,term,offset):
  return round(df['close'].rolling(term).mean().shift(offset),1)


# 320最高値
def highest(df, term):
  return round(df['high'].rolling(don_buy_term).max().shift(1),1)


# 320最安値
def lowest(df, term):
  return round(df['low'].rolling(don_buy_term).min().shift(1),1)

In [19]:
# ------------------------------
# 400ポジション確認
# ------------------------------
def check_positions(flag):
	if backtest: # バックテスト設定
		if flag["position"]["exist"] == True: # 直前にポジションがある場合は保有期間を加算
			flag["position"]["count"] += 1
	else:
		failed = 0
		while True:
			try:
				positions = bitmex.private_get_getpositions(params={'product_code':'FX_BTC_JPY'})
				if positions:
					for pos in positions:
						if pos["symbol"] == "XBTUSD":
							if not pos["currentQty"] == 0:
								if pos["currentQty"] > 0:
									side = "buy"
								else:
									side = "sell"
								if flag["position"]["exist"] == False: # ファーストエントリの場合はポジション数を１に変更する
									flag["add-position"]["count"] = 1
								if flag["position"]["exist"] == True: # 直前にポジションがある場合は保有期間を加算
									flag["position"]["count"] += 1
								lot = int(np.abs(pos["currentQty"])) # 絶対値を返す
								price = pos["avgEntryPrice"]
								exist = True
								date = pos["timestamp"] # 最終ポジションの日時に上書き
							else:
								price = 0
								lot = 0
								side = None
								exist = False
								date = None
				else:
					price = 0
					lot = 0
					side = None
					exist = False
					date = None
				# ポジションフラグ更新
				flag["position"]["exist"] = exist
				flag["position"]["side"] = side
				flag["position"]["final_price"] = price
				flag["position"]["total_lot"] = lot
				flag["position"]["first_date"] = date # 本番運用時は、最終ポジションのエントリー日時になることに注意
				if flag["add-position"]["count"] >= 1 and flag["add-position"]["unit-size"] == 0: # 1回目ポジションを持った状態で中断/再開した場合は、エントリー回数を上限値に更新する。
					flag["add-position"]["count"] = entry_times
				return price,lot,side
			except ccxt.BaseError as e:
				failed += 1
				if failed > 10:
					print_log("ポジション取得APIでエラーに10回失敗しました : " + str(e))
					print_log(f"{error_recovery_time}秒待機してやり直します")
				time.sleep(error_recovery_time)

In [40]:
# ------------------------------
# 500エントリー判定
# ------------------------------
def entry(last_data, flag):
	if flag["position"]["exist"]:
		return flag
	# 条件判定
	signal = _donchian_signal(last_data)
	if not signal["side"] == None:
		print_log(signal["log"])
		# フィルター条件を確認
		if _sma_filter(last_data, signal) == False:
			print_log("フィルターのエントリー条件を満たさなかったため、エントリーしません")
			return flag
		# ロット計算
		flag = _calculate_lot(flag, last_data, signal)
		if flag["add-position"]["unit-size"] >= 0.0025 * last_data.iloc[-1]["close"]:
			print_log(f"{flag['add-position']['count'] + 1}回目のポジションを、{last_data.iloc[-1]['close']}USDあたりに{round(flag['add-position']['unit-size']/last_data.iloc[-1]['close'],4)}XBT={flag['add-position']['unit-size']}ロット注文します")
			# 指値注文
			price = _limit(signal["side"].lower(), flag["add-position"]["unit-size"])["average"] if backtest == False else last_data.iloc[-1]['close']  # バックテスト設定
			# バックテストの場合はポジション確認をしないので注文時にフラグを更新する　バックテスト設定
			if backtest:
				flag["position"]["exist"] = True
				flag["position"]["side"] = signal["side"]
				flag["position"]["first_date"] = last_data.iloc[-1]["datetime"]
				flag["position"]["first_price"] = price
				flag["position"]["final_price"] = price
				flag["position"]["total_lot"] = flag["add-position"]["unit-size"]
				flag["position"]["total_value"] = round(flag["add-position"]["unit-size"] / price, 4)
				flag["position"]["positions"].append({
						"count":flag['add-position']['count'] + 1,
						"date":last_data.iloc[-1]["datetime"],
						"side":signal["side"],
						"price":price,
						"lot":flag["add-position"]["unit-size"],
						"value":round(flag["add-position"]["unit-size"] / price, 4),
	          "commision_usd":np.floor(-0.00025 * flag["add-position"]["unit-size"]),
            "commision_xbt":round(-0.00025 * flag["add-position"]["unit-size"] / price, 4),
				})
				flag["add-position"]["count"] = 1
		else:
			print_log(f"注文可能枚数{lot}が、最低注文単位に満たなかったので注文を見送ります")
	return flag


# 510条件判定_ドンチアンブレイク
def _donchian_signal(last_data):
	highest = last_data.iloc[-1 * don_buy_term:-2]["high"].max()
	# print(f"最高値は{highest}です")
	if last_data.iloc[-1][judge_price["buy"]] > highest:
		log = f"過去{don_buy_term}足の最高値{highest}USDを、直近の{judge_price['buy']}値が{last_data.iloc[-1][judge_price['buy']]}USDでブレイクしました"
		return {"side":"buy", "price":last_data.iloc[-1]["close"], "log":log}
	lowest = last_data.iloc[-1 * don_sell_term:-2]["low"].min()
	# print(f"最安値は{lowest}です")
	if last_data.iloc[-1][judge_price["sell"]] < lowest:
		log = f"過去{don_sell_term}足の最安値{lowest}USDを、直近の{judge_price['sell']}値が{last_data.iloc[-1][judge_price['sell']]}USDでブレイクしました"
		return {"side":"sell","price":last_data.iloc[-1]["close"], "log":log}
	return {"side" : None , "price":0, "log":None}


# 511条件判定_移動平均フィルタ
def _sma_filter(last_data, signal):
	if sma_filter == "OFF":
		return True
	if len(last_data) < sma_term*alfa+sma_offset*alfa:
		return True
	if signal["side"] == "buy" and last_data.iloc[-1]["close"] > last_data.iloc[-1]["sma"]:
		return True
	if signal["side"] == "sell" and last_data.iloc[-1]["close"] < last_data.iloc[-1]["sma"]:
		return True
	return False


# 520ロット計算
def _calculate_lot(flag, last_data, signal):
	if flag["add-position"]["count"] == 0: # 第一エントリーの場合
		# 口座残高を取得する
		balance = _collateral() if backtest == False else flag["balance"]["collateral"] # 0.0001BTXから成るBTX値を返す バックテスト設定
		# ATRを計算する
		atr = _atr(last_data)
		# 損切幅を計算する
		stop_range = atr_range * atr if atr > 0 else 1 # 損切幅が0の場合は1USDで代替する（ロット計算のエラー防止）
		# 資金、リスク率、損切幅からロット数を計算する
		lot = np.floor(balance * trade_risk * last_data.iloc[-1]["close"] * (last_data.iloc[-1]["close"] - stop_range) / stop_range)
		flag["add-position"]["unit-size"] = int(np.floor(lot / entry_times))
		flag["add-position"]["unit-range"] = atr * entry_range
		flag["position"]["stop_range"] = stop_range
		flag["position"]["atr"] = atr
		print_log(f"許容リスク{trade_risk * 100}%と損切幅{np.floor(stop_range)}USDに基づく注文数は{round(lot/last_data.iloc[-1]['close'],4)}XBT={np.floor(lot)}ロットです。")
		print_log(f"{entry_times}回に分けて、{np.floor(flag['add-position']['unit-range'])}USDごとに、{round(flag['add-position']['unit-size']/last_data.iloc[-1]['close'],4)}XBT={flag['add-position']['unit-size']}ロットずつ注文します")
	return flag


# 521口座残高取得
def _collateral():
	while True:
		try:
			collateral = bitmex.fetch_balance()
			spendable_collateral = collateral["free"]["BTC"]
			print_log(f"現在のアカウント残高は{round(collateral['total']['BTC'],4)}XBT={round(collateral['total']['BTC'] * last_data.iloc[-1]['close'])}ロットです")
			print_log(f"新規注文に利用可能な証拠金の額は{round(float(spendable_collateral),4)}XBT={round(float(spendable_collateral) * last_data.iloc[-1]['close'])}ロットです")
			return float(spendable_collateral)
		except ccxt.BaseError as e:
			print_log("口座残高取得API通信に失敗しました ： " + str(e))
			print_log(f"{error_recovery_time}秒待機してやり直します")
			time.sleep(error_recovery_time)


# 522ATR計算
def _atr(last_data):
	high_sum = last_data.iloc[-1 * atr_term:]["high"].sum()
	low_sum  = last_data.iloc[-1 * atr_term:]["low"].sum()
	atr = (high_sum - low_sum) / atr_term
	print_log(f"現在の{atr_term}期間のATRは{np.floor(atr)}USDです")
	return atr


# 530指値注文
def _limit(side,lot):
	while True:
		data = get_price(limit=2) # 直前の価格を取得する。2つ以上取得することで未確定足を取得することができる。
		try:
			order = bitmex.create_order(
				symbol = 'BTC/JPY',
				type='limit', # 指値
				side= side,
				amount= lot,
				price= data.iloc[-1]['close']-1 if side=='buy' else data.iloc[-1]['close']+1, # 直前の不確定足の終値に対して１USD幅で指値注文
				params = { "product_code" : "FX_BTC_JPY" }
			)
			flag["order"]["exist"] = True
			print_log("--------------------")
			print_log(f"{order['datetime']}  サイド：{side.upper()} 指値:{round(order['price'],2)}  注文数:{np.floor(order['amount'])}  で注文しました。約定状況は{order['status']}です。")
			print_log("--------------------")
			return order
		except ccxt.BaseError as e:
			print_log("注文APIでエラー発生 : " + str(e))
			print_log(f"{error_recovery_time}秒待機してやり直します")
			time.sleep(error_recovery_time)

In [21]:
# ------------------------------
# 600損切判定
# ------------------------------
def stop_loss(last_data, flag):
	# トレ-リングストップを実行
	if trailing_config == "ON":
		flag = _trailing_stop(last_data, flag)
	# 暫定損益計算
	pnl = _temporal_pnl(last_data,flag)
	# 損切判定
	signal = _stop_loss_signal(last_data, flag)
	if not signal["side"] == None:
		lot = flag["position"]["total_lot"]
		side = signal["side"]
		print_log(signal["log"])
		print_log(f"{last_data.iloc[-1]['close']}USDあたりに{lot/last_data.iloc[-1]['close']}XBT={lot}ロットで{side.upper()}の成行注文を出してポジションを決済します。")
		# 成行注文
		close_price = _market(side,lot)["average"] if backtest == False else last_data.iloc[-1]["close"]
		# 損益記録
		if backtest: # バックテスト設定
			records(flag, last_data, close_price)
		# フラグ更新
		flag["position"]["exist"] = False
		flag["position"]["side"] = None
		flag["position"]["first_date"] = None
		flag["position"]["first_price"] = 0
		flag["position"]["final_price"] = 0
		flag["position"]["count"] = 0
		flag["position"]["stop-AF"] = stop_AF
		flag["position"]["stop-EP"] = 0
		flag["position"]["total_lot"] = 0
		flag["position"]["total_value"] = 0
		flag["position"]["positions"] =[]
		flag["add-position"]["count"] = 0
	return flag


# 610暫定損益計算
def _temporal_pnl(last_data,flag):
  if backtest: # バックテスト設定
    return last_data.iloc[-1]["close"] * (1 - 0.00075) - flag["position"]["first_price"] if flag["position"]["side"] == "buy" else flag["position"]["first_price"] - last_data.iloc[-1]["close"] * (1 + 0.00075) # 成行注文手数料を加えた終値と注文価格を比較する
  else:
    while True:
      try:
        pos = bitmex.private_get_position()
        commission = 0.00075 * flag["position"]["total_lot"] / last_data.iloc[-1]["close"] * 2 # エントリーと決済で成行注文したと仮定して手数料を計算
        unrealisedPnlPcnt = pos[0]["unrealisedPnlPcnt"] * 100
        unrealisedRoePcnt = pos[0]["unrealisedRoePcnt"] * 100
        pnl = pos[0]["unrealisedPnl"] / 100000000
        adjust_pnl = pnl - commission
        print_log(f"損益:{round(unrealisedPnlPcnt,2)}% ROE:{round(unrealisedRoePcnt,2)}% 手数料:{round(commission,4)}XBT 未実現損益:{round(pnl,4)}XBT 未実現損益-手数料:{round(adjust_pnl,4)}XBT")
        return adjust_pnl
      except ccxt.BaseError as e:
        print_log("ポジション取得APIで失敗しました : " + str(e))
        print_log(f"{error_recovery_time}秒待機してやり直します")
        time.sleep(error_recovery_time)


# 620トレーリングストップ
def _trailing_stop(last_data, flag):
	# 未追加のポジションがあれば何もしない
	if flag["add-position"]["count"] < entry_times:
		return flag
	# 高値／安値がエントリー価格からいくら離れたか計算
	if flag["position"]["side"] == "buy":
		moved_range = round(last_data.iloc[-1]["high"] - flag["position"]["final_price"])
	if flag["position"]["side"] == "sell":
		moved_range = round(flag["position"]["final_price"] - last_data.iloc[-1]["low"])
	# 最高値・最安値を更新したか調べる
	if moved_range < 0 or moved_range <= flag["position"]["stop-EP"]:
		return flag
	else:
		flag["position"]["stop-EP"] = moved_range
	# 加速係数に応じて損切りラインを動かす
	flag["position"]["stop_range"] = round(flag["position"]["stop_range"] - (moved_range + flag["position"]["stop_range"]) * flag["position"]["stop-AF"])
	# 加速係数を更新
	flag["position"]["stop-AF"] = round(flag["position"]["stop-AF"] + stop_AF_add ,2)
	if flag["position"]["stop-AF"] >= stop_AF_max:
		flag["position"]["stop-AF"] = stop_AF_max
	# ログ出力
	if flag["position"]["side"] == "buy":
		print_log(f"トレイリングストップの発動：ストップ位置を{round(flag['position']['final_price'] - flag['position']['stop_range'])}USDに動かして、加速係数を{flag['position']['stop-AF']}に更新します")
	else:
		print_log(f"トレイリングストップの発動：ストップ位置を{round(flag['position']['final_price'] + flag['position']['stop_range'])}USDに動かして、加速係数を{flag['position']['stop-AF']}に更新します")
	return flag


#630損切りシグナル
def _stop_loss_signal(last_data, flag):
	if flag["position"]["side"] == "buy":
		stop_price = flag["position"]["final_price"] - flag["position"]["stop_range"]
		if last_data.iloc[-1]["close"] < stop_price:
			log = f"{stop_price}USDの損切ラインに引っかかりました。"
			signal = {"side":"sell", "log":log}
			return signal
	if flag["position"]["side"] == "sell":
		stop_price = flag["position"]["final_price"] + flag["position"]["stop_range"]
		if last_data.iloc[-1]["close"] > stop_price:
			log = f"{stop_price}USDの損切ラインに引っかかりました。"
			signal = {"side":"buy", "log":log}
			return signal
	signal = {"side":None, "log":None}
	return signal

In [22]:
# ------------------------------
# 700ドテン判定
# ------------------------------
def doten(last_data, flag):
  if not flag["position"]["exist"]:
    return flag
  signal = _donchian_signal(last_data)
  if flag["position"]["side"] == "buy" and signal["side"] == "sell" or flag["position"]["side"] == "sell" and signal["side"] == "buy":
    lot = flag["position"]["total_lot"]
    side = signal["side"]
    print_log(signal["log"])
    print_log(f"{last_data.iloc[-1]['close']}USDあたりに{lot/last_data.iloc[-1]['close']}XBT={lot}ロットで{side.upper()}の成行注文を出してポジションを決済します。")
    # 成行注文
    closed_price = _market(side,lot)["average"] if backtest == False else last_data.iloc[-1]["close"]
    # 損益記録
    if backtest: # バックテスト設定
      records(flag, last_data, closed_price)
    # フラグ更新
    flag["position"]["exist"] = False
    flag["position"]["side"] = None
    flag["position"]["final_price"] = 0
    flag["position"]["first_date"] = None
    flag["position"]["first_price"] = 0
    flag["position"]["count"] = 0
    flag["position"]["stop-AF"] = stop_AF
    flag["position"]["stop-EP"] = 0
    flag["position"]["total_lot"] = 0
    flag["position"]["total_value"] = 0
    flag["position"]["positions"] =[]
    flag["add-position"]["count"] = 0
    # ドテン注文
    print_log("ドテン注文します")
    flag = entry(last_data, flag)
  return flag


# 710成行注文
def _market(side,lot):
  while True:
    try:
      order = bitmex.create_order(
        symbol = 'BTC/USD',
        type = 'market', # 成行
        side = side,
        amount = lot,
      )
      print_log("--------------------")
      print_log(f"{order['datetime']}  サイド：{side.upper()} 成行:{order['price']}  注文数:{order['amount']}で注文しました。約定状況は{order['status']}です。")
      print_log("--------------------")
      return order
    except ccxt.BaseError as e:
      print_log("注文APIでエラー発生 : " + str(e))
      print_log(f"{error_recovery_time}秒待機してやり直します")
      time.sleep(error_recovery_time)

In [23]:
# ------------------------------
# 800追加エントリー判定
# ------------------------------
def add_position(last_data, flag):
  # ポジションがない場合は何もしない
  if flag["position"]["exist"] == False:
    return flag
  # 以下の場合は、追加ポジションを取らない
  if flag["add-position"]["count"] >= entry_times:
    return flag
  signal = _add_position_signal(last_data, flag)
  if not signal["side"] == None:
    lot = flag["add-position"]["unit-size"]
    side = signal["side"]
    print_log(signal["log"])
    print_log(f"{flag['add-position']['count'] + 1}/{entry_times}回目の追加注文を出します")
    # 成行注文
    closed_price = _market(side.lower(),lot)["average"] if backtest == False else last_data.iloc[-1]["close"]
    if backtest:
      flag["position"]["final_price"] = closed_price # 最新のポジションの価格で上書き
      flag["position"]["total_lot"] += lot # 追加ポジション分のロットを加算
      flag["position"]["total_value"] += round(flag["add-position"]["unit-size"] / closed_price, 4)
      flag["position"]["positions"].append({
          "count":flag['add-position']['count'] + 1,
          "date":last_data.iloc[-1]["datetime"],
          "side":signal["side"],
          "price":closed_price,
          "lot":lot,
          "value":round(lot / closed_price, 4),
          "commision_usd":np.floor(0.00075 * lot),
          "commision_xbt":round(0.00075 * lot / closed_price, 4),
      })
    # ポジション数を加算する
    flag["add-position"]["count"] += 1
    stop_price = flag['position']['final_price'] - flag["position"]["stop_range"] if signal["side"] == "buy" else flag['position']['final_price'] + flag["position"]["stop_range"]
    print_log(f"ストップ位置を{stop_price}USDに更新しました")
    return flag
  return flag


# 810追加エントリーシグナル
def _add_position_signal(last_data, flag):
  if flag["position"]["side"] == "buy" and last_data.iloc[-1]["close"] - flag["position"]["final_price"] > flag["add-position"]["unit-range"]:
    log = f"前回のエントリー価格{flag['position']['final_price']}USDからブレイクアウトの方向に{flag['add-position']['unit-range']}USD以上動きました"
    signal = {"side":"buy", "log":log}
    return signal
  if flag["position"]["side"] == "sell" and flag["position"]["final_price"] - last_data.iloc[-1]["close"] > flag["add-position"]["unit-range"]:
    log = f"前回のエントリー価格{flag['position']['final_price']}USDからブレイクアウトの方向に{flag['add-position']['unit-range']}USD以上動きました"
    signal = {"side":"sell", "log":log}
    return signal
  signal = {"side":None, "log":None}
  return signal

In [24]:
# ------------------------------
# 900成績表示
# ------------------------------
# 910成績記録 #決済の直後に挿入して使用する
def records(flag, last_data, closed_price):
	# 手数料の記録
	entry_commisions_usd = 0
	entry_commisions_xbt = 0
	for c in flag["position"]["positions"]:
		entry_commisions_usd += c["commision_usd"] # 手数料[USD]
		entry_commisions_xbt += c["commision_xbt"] # 手数料[XBT]
	exit_commisions_usd = 0.00075 * flag["position"]["total_lot"] # 手数料[USD]
	commisions_usd = np.floor(entry_commisions_usd + exit_commisions_usd) # 手数料[USD]
	exit_commisions_xbt = 0.00075 * flag["position"]["total_lot"] / closed_price # 手数料[XBT]
	commisions_xbt = round(entry_commisions_xbt + exit_commisions_xbt, 4) # 手数料[XBT]
	flag["records"]["commisions"].append(commisions_usd)
	# 決済日時、価格、保有期間を記録
	flag["records"]["exit_date"].append(last_data.iloc[-1]['datetime'])
	flag["records"]["exit_price"].append(closed_price)
	flag["records"]["holding-periods"].append(flag["position"]["count"])
	# 損益(USD)の計算
	buy_profit_usd = 0
	for i in flag["position"]["positions"]:
		buy_profit_usd += (closed_price - i["price"]) * i["lot"] / i["price"]
	sell_profit_usd = 0
	for i in flag["position"]["positions"]:
		sell_profit_usd += (i["price"] - closed_price) * i["lot"] / i["price"]
	# 損益(BTX)の計算
	buy_profit_xbt = flag["position"]["total_value"] - flag["position"]["total_lot"] / closed_price
	sell_profit_xbt = flag["position"]["total_lot"] / closed_price - flag["position"]["total_value"]
	# ロングエントリーの場合の記録
	if flag["position"]["side"] == "buy":
		# 損益の記録
		flag["records"]["side"].append("buy")
		flag["records"]["profit_usd"].append(buy_profit_usd - commisions_usd)
		flag["records"]["profit_xbt"].append(buy_profit_xbt - commisions_xbt)
		flag["records"]["return"].append(round(buy_profit_usd / flag["position"]["first_price"] * 100))
		# 資産の更新記録
		flag["balance"]["collateral"] += (buy_profit_xbt - commisions_xbt)
		flag["records"]["collateral"].append(flag["balance"]["collateral"])
	# ショートエントリーの場合の記録
	if flag["position"]["side"] == "sell":
		# 損益の記録
		flag["records"]["side"].append("sell")
		flag["records"]["profit_usd"].append(sell_profit_usd - commisions_usd)
		flag["records"]["profit_xbt"].append(sell_profit_xbt - commisions_xbt)
		flag["records"]["return"].append(round(sell_profit_usd / flag["position"]["first_price"] * 100))
		# 資産の更新記録
		flag["balance"]["collateral"] += (sell_profit_xbt - commisions_xbt)
		flag["records"]["collateral"].append(flag["balance"]["collateral"])
	# エントリー情報の記録
	flag["records"]["entry_date"].append(flag["position"]["first_date"])
	flag["records"]["entry_price"].append(flag["position"]["first_price"])
	flag["records"]["lot"].append(flag["position"]["total_lot"])
	return flag


# 920成績表示
def order_history(flag):
	# 注文履歴を記録したpandas DataFrameを作成
	order_history = pd.DataFrame({
		"Entry_date"  :  flag["records"]["entry_date"],
		"Entry_price" :  flag["records"]["entry_price"],
		"Side"        :  flag["records"]["side"],
		"Lot"         :  flag["records"]["lot"],
		"Exit_date"   :  flag["records"]["exit_date"],
		"Exit_price"  :  flag["records"]["exit_price"],
		"Profit"      :  flag["records"]["profit_usd"],
		"Profit_xbt"  :  flag["records"]["profit_xbt"],
		"Rate"        :  flag["records"]["return"],
		"Periods"     :  flag["records"]["holding-periods"],
		"Commisions"  :  flag["records"]["commisions"],
		"Collateral"  :  flag["records"]["collateral"]
	})
	# 総損益の列を追加する
	order_history["Gross"] = order_history.Profit.cumsum()
	# 最大ドローダウンの列を追加する
	order_history["Drawdown"] = order_history.Gross.cummax().subtract(order_history.Gross)
	order_history["DrawdownRate"] = round(order_history.Drawdown / order_history.Gross.cummax() * 100,1)
	try:
		print("バックテストの結果")
		print("-----------------------------------")
		print("総合の成績")
		print("-----------------------------------")
		print(f"全トレード数       :  {len(order_history)}回")
		print(f"勝率               :  {round(len(order_history[order_history.Profit>0]) / len(order_history) * 100,1)}％")
		print(f"平均リターン       :  {round(order_history.Rate.mean(),2)}％")
		print(f"平均保有期間       :  {round(order_history.Periods.mean(),1)}足分")
		print("")
		print(f"最大の勝ちトレード :  {round(order_history.Profit.max()):,} USD")
		print(f"最大の負けトレード :  {round(order_history.Profit.min()):,} USD")
		print(f"最大ドローダウン   :  {round(-1 * order_history.Drawdown.max()):,} USD / {-1 * order_history.DrawdownRate.loc[order_history.Drawdown.idxmax()]}％")
		print(f"利益合計           :  {round(order_history[order_history.Profit>0].Profit.sum()):,} USD")
		print(f"損失合計           :  {round(order_history[order_history.Profit<0].Profit.sum()):,} USD")
		print("")
		print(f"最終損益           :  {round(order_history.Profit.sum()):,} USD")
		print(f"手数料合計         :  {round(-1 * order_history.Commisions.sum()):,} USD")
		print(f"プロフィットファクター :  {round( -1 * (order_history[order_history.Profit>0].Profit.sum() / order_history[order_history.Profit<0].Profit.sum()) ,2)}")
	except:
		print(f"決済されたトレードは現在ありません")
	return order_history

In [25]:
# 930グラフ表示
def fig_show(order_history):
  # 複数のグラフ用の枠をセットする
  fig = make_subplots(
      rows=5,
      cols=1,
      shared_xaxes=True,
      vertical_spacing=0.04,
      specs=[[{}], # 1行目
             [{}], # 2行目
             [{"rowspan": 3}], # 3行目（3段使用）
             [None], # 4行目なし
             [None]], # 4行目なし
      )
  # 全体レイアウト
  fig.update_layout(
      height=1500,
  )
  # 1行目の枠に資産推移を追加
  fig.append_trace(go.Scatter(
      x=order_history.Exit_date,
      y=order_history.Collateral,
      mode='lines',
      line=dict(color='#16b7c5', width=2),
      name="資産推移"
      ),
      row=1,
      col=1
  )
  # 2行目の枠に累積損益の推移を追加
  fig.append_trace(go.Scatter(
      x=order_history.Exit_date,
      y=order_history.Gross,
      mode='lines',
      line=dict(color='#1AAE9F', width=2),
      name="累積損益推移"
      ),
      row=2,
      col=1
  )
  # 3行目の枠にローソクチャートを追加
  fig.append_trace(go.Candlestick(
      x=last_data['datetime'],
      open=last_data['open'],
      high=last_data['high'],
      low=last_data['low'],
      close=last_data['close'],
      increasing_line_color= '#e73b3a',
      decreasing_line_color= '#03c103',
      name=timeframe+'足',
      ),
      row=3,
      col=1,
  )
  # ローソクチャートにインジケーターを追加
   #　移動平均線
  if 'sma' in last_data.columns.to_list():
    fig.add_trace(go.Scatter(
        x=last_data.datetime,
        y=last_data.sma,
        mode='lines',
        line=dict(color='#E3615F', width=1),
        name=f"移動平均線({sma_term*alfa}期間：オフセット{sma_offset*alfa}期間)"
        ),
        row=3,
        col=1
    )
  #　最高値
  if 'highest' in last_data.columns.to_list():
    fig.add_trace(go.Scatter(
        x=last_data.datetime,
        y=last_data.highest,
        mode='lines',
        line=dict(color='#94f996', width=1),
        name=f"最高値ライン({don_buy_term}期間)"
        ),
        row=3,
        col=1
    )
  #　最安値
  if 'lowest' in last_data.columns.to_list():
    fig.add_trace(go.Scatter(
        x=last_data.datetime,
        y=last_data.lowest,
        mode='lines',
        line=dict(color='#16b7c5', width=1),
        name=f"最安値ライン({don_sell_term}期間)"
        ),
        row=3,
        col=1
    )
  # ロングエントリーマーカー
  buy = order_history[order_history['Side'] == 'buy']
  fig.add_trace(go.Scatter(
      x=buy.Entry_date,
      y=buy.Entry_price,
      mode='markers',
      marker=dict(symbol='triangle-right',color='#2A89DC',size=10),
      name="ロングエントリー"
      ),
      row=3,
      col=1
  )
  # ショートエントリーマーカー
  sell = order_history[order_history['Side'] == 'sell']
  fig.add_trace(go.Scatter(
      x=sell.Entry_date,
      y=sell.Entry_price,
      mode='markers',
      marker=dict(symbol='triangle-right',color='#D3455B',size=10),
      name="ショートエントリー"
      ),
      row=3,
      col=1
  )
  # 決済マーカー
  fig.add_trace(go.Scatter(
      x=order_history.Exit_date,
      y=order_history.Exit_price,
      mode='markers',
      marker=dict(symbol='triangle-left',color='#BD34D1', size=10),
      name="決済"
      ),
      row=3,
      col=1
  )
  # グラフ表示
  fig.show()

In [26]:
# ------------------------------
# 000-B初号機V1.0（バックテスト用）メイン処理
# ------------------------------
if backtest:
  # # バックテスト用メイン設定--------------------------
  # timeframe = "1h"               # 時間足
  # limit = 10000                   # データ数
  # collateral = 0.1               # バックテスト時の資産 単位：XBT
  # trade_risk = 0.03              # 1トレードあたり口座の何％まで損失を許容するか
  # backtest_print_log = "OFF"     # バックテスト時のログ表示設定 "ON" or "OFF"
  # # ドンチアンブレイクの設定--------------------------
  # don_buy_term = 30              # 最高値（上値）ブレイクアウト期間
  # don_sell_term = 30             # 最安値（下値）ブレイクアウト期間
  # judge_price={
  #   "buy" : "high",              # ブレイク判断　高値（high)か終値（close）を使用
  #   "SELL": "low",               # ブレイク判断　安値 (low)か終値（close）を使用
  # }
  # sma_filter = "ON"              # 移動平均線のフィルターの適用 "ON" or "OFF"
  # # ポジション追加の設定----------------------------
  # entry_times = 2                # 追加ポジションを持つ回数
  # entry_range = 1                # 何ATR幅ごとに追加ポジションを持つか
  # # 損切り設定--------------------------------------
  # atr_term = 5                   # ATRの期間
  # atr_range = 2                  # 損切りに使用するATRの幅
  # # トレイリングストップ設定------------------------
  # trailing_config = "ON"         # ONで有効 OFFで無効
  # stop_AF = 0.02                 # 加速係数
  # stop_AF_add = 0.02             # 加速係数を増やす度合
  # stop_AF_max = 0.3              # 加速係数の上限
  # # トレンドフィルター用----------------------------
  # last_data = pd.DataFrame()
  # alfa = 1                       #価格データの時間足（timeframe)と異なる時間足のインジケーターを使用する場合。例：価格データ15分足、インジケーター45分足の場合は alfa = 3
  # sma_term = 200                 #移動平均期間
  # sma_offset = 0                 #移動平均線を移動する期間（プラス：右へ移動 マイナス：左へ移動）
  # #-------------------------------------------------
  need_term = max(don_buy_term, don_sell_term, sma_term*alfa+sma_offset*alfa)+1
  flag = {
    "order":{
      "exist": False,
    },
    "position":{
      "exist" : False,
      "side" : "",
      "first_date" :"",
      "first_price" :0,
      "final_price": 0,
      "count":0,
      "stop_range":0,
      "atr":0,
      "stop-AF": stop_AF,
      "stop-EP":0,
      "positions": [],
      "total_lot": 0,
      "total_value":0,
      "total_entry_commisions":0,
    },
    "add-position":{
      "count":0,
      "unit-size":0,
      "unit-range":0,
    },
    "records":{
    "entry_date":[],
    "exit_date":[],
    "entry_price":[],
    "exit_price":[],
    "lot":[],
    "profit_usd":[],
    "profit_xbt":[],
    "return":[],
    "side":[],
    "holding-periods":[],
    "commisions":[],
    "collateral":[]
    },
    "balance":{
    "collateral":collateral,
    },
  }
  # 200価格データ取得　テストデータを作成
  test_data = get_price(timeframe=timeframe, limit=limit)
  # 300インジケーター作成
  test_data['sma'] = sma(df=test_data,term=sma_term*alfa,offset=sma_offset*alfa)
  test_data['highest'] = highest(df=test_data,term=don_buy_term)
  test_data['lowest'] = lowest(df=test_data,term=don_sell_term)
  print("--------------------------")
  print("テスト期間：")
  print("開始時点 : " + str(test_data.iloc[0]["datetime"]))
  print("終了時点 : " + str(test_data.iloc[-1]["datetime"]))
  print(f"{timeframe}足　件数 : {str(len(test_data))}件")
  print(f"初期資金 : {collateral}XBT")
  print(f"トレードリスク : {trade_risk * 100}%")
  print("パラメーター設定----------")
  print(f"最高値（上値）ブレイクアウト判定値 : {judge_price['buy']}値 期間 : {don_buy_term}期間")
  print(f"最安値（下値）ブレイクアウト判定値 : {judge_price['sell']}値 期間 : {don_sell_term}期間")
  print(f"ATR期間 : {atr_term}期間")
  print(f"損切幅 : {atr_range}ATR")
  print(f"トレイリングストップ : {trailing_config}")
  print(f"SMAフィルター : {sma_filter}")
  if sma_filter == "ON":
    print(f"移動平均 : {sma_term}期間")
    print(f"alfa : {alfa}")
    print(f"移動平均オフセット : {sma_offset}期間")
  # 判断用の積算データフレームを作成
  last_data = pd.DataFrame()
  i = 0
  while i < len(test_data):
    # エントリー条件の判定使う過去ｎ足分のデータを準備する
    if len(last_data) < need_term:
      last_data = last_data.append(test_data.iloc[i])
      i += 1
      continue
    print_price(last_data.iloc[-1])
    # 400ポジション確認
    check_positions(flag)
    # ポジションがある場合
    if flag["position"]["exist"]:
    # 600損切判定
      flag = stop_loss(last_data, flag)
    # 700ド点判定
      flag = doten(last_data, flag)
    # 800追加エントリー判定
      flag = add_position(last_data, flag)
    # ポジションがない場合
    else:
    # 500エントリー判定
      flag = entry(last_data,flag)
    last_data = last_data.append(test_data.iloc[i])
    i += 1
  # 900成績表示
  oh = order_history(flag)
  pd.options.display.max_rows = None #行を省略しないで表示
  display(oh)
  fig_show(oh)

In [39]:
# ------------------------------
# 000-F初号機V1.0（本番用）メイン処理
# ------------------------------
if not backtest:
  flag = {
    "order":{
      "exist": False,
    },
    "position":{
      "exist" : False,
      "side" : "",
      "first_date" :"",
      "first_price" :0,
      "final_price": 0,
      "count":0,
      "stop_range":0,
      "atr":0,
      "stop-AF": stop_AF,
      "stop-EP":0,
      "positions": [],
      "total_lot": 0,
      "total_value":0,
      "total_entry_commisions":0,
    },
    "add-position":{
      "count":0,
      "unit-size":0,
      "unit-range":0,
    },
    "records":{
    "entry_date":[],
    "exit_date":[],
    "entry_price":[],
    "exit_price":[],
    "lot":[],
    "profit_usd":[],
    "profit_xbt":[],
    "return":[],
    "side":[],
    "holding-periods":[],
    "commisions":[],
    "collateral":[]
    },
    "balance":{
    "collateral":collateral,
    },
  }
  # 最低限、保持が必要なローソク足の期間を準備
  # need_term = max(don_buy_term, don_sell_term, sma_term*alfa+sma_offset*alfa)+1
  need_term = 10
  last_data = get_price(timeframe=timeframe,limit=need_term)
  print_log(f"{need_term}期間分のデータの準備中")
  last_data = last_data[-1*need_term-2:-2]
  print(f"last data: {last_data}")
  print_log("---実行開始---")
  while True:
    # 100約定確認
    check_open_order()
    # 200価格データ取得
    data = get_price(timeframe=timeframe,limit=2) # to be changed to gent realohlcv
    print(f"data: {data}")
    print(f"last data -1: {last_data.iloc[-1]['timestamp']}")
    print(f"data -2: {data.iloc[-2]['timestamp']}")
    if data.iloc[-2]["timestamp"] > last_data.iloc[-1]["timestamp"]:
      last_data = last_data.append(data.iloc[-2],ignore_index = True)
      # 300インジケーター作成
      last_data['sma'] = sma(df=last_data,term=sma_term*alfa,offset=sma_offset*alfa)
      last_data['highest'] = highest(df=last_data,term=don_buy_term)
      last_data['lowest'] = lowest(df=last_data,term=don_sell_term)
      # 確定足を表示
      print_price(last_data.iloc[-1])
      # 400ポジション確認
      # ポジション有りの場合
      averageprice,lot,size = check_positions(flag)
      if not size == None: # ポジション有りの場合
      # # 600損切判定
        flag = stop_loss(last_data, flag)
      # # 700ド点判定
        flag = doten(last_data, flag)
      # # 800追加エントリー判定
        flag = add_position(last_data, flag)
      # # ポジションがない場合
      else:
      # # 500エントリー判定
        flag = entry(last_data,flag)
      # 必要期間分以上のデータを削除
      if len(last_data) > need_term:
        last_data = last_data[1:]
    time.sleep(wait)

2023-11-14T17:16:18.037
2023-11-14T17:12:45.16
2023-11-14T17:07:46.777
10期間分のデータの準備中
last data:                     datetime      timestamp       open       high        low  \
3  2023-11-15 11:11:00+09:00  1700014260000  5459997.0  5466385.0  5459997.0   
4  2023-11-15 11:12:00+09:00  1700014320000  5456227.0  5460707.0  5456227.0   
5  2023-11-15 11:13:00+09:00  1700014380000  5456592.0  5458329.0  5452079.0   
6  2023-11-15 11:14:00+09:00  1700014440000  5460343.0  5463979.0  5455233.0   
7  2023-11-15 11:15:00+09:00  1700014500000  5458643.0  5458825.0  5455787.0   
8  2023-11-15 11:16:00+09:00  1700014560000  5455626.0  5460717.0  5455076.0   
9  2023-11-15 11:17:00+09:00  1700014620000  5451812.0  5455065.0  5451812.0   
10 2023-11-15 11:18:00+09:00  1700014680000  5451539.0  5452228.0  5450000.0   

        close    volume  
3   5466048.0  0.544641  
4   5459997.0  1.221855  
5   5456912.0  0.677901  
6   5455420.0  0.507707  
7   5458817.0  0.533812  
8   5458456.0  0.429373  
9

<ipython-input-39-278bbe346728>:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  last_data = last_data.append(data.iloc[-2],ignore_index = True)


過去10足の最安値5451812.0USDを、直近のlow値が5450000.0USDでブレイクしました
現在のアカウント残高は0.0655XBT=357089ロットです
新規注文に利用可能な証拠金の額は0.0655XBT=357089ロットです
現在の5期間のATRは3139.0USDです
許容リスク3.0%と損切幅6279.0USDに基づく注文数は1.704XBT=9289317.0ロットです。
2回に分けて、3139.0USDごとに、0.852XBT=4644658ロットずつ注文します
1回目のポジションを、5451539.0USDあたりに0.852XBT=4644658ロット注文します
2023-11-14T17:16:18.707
注文APIでエラー発生 : bitflyer POST https://api.bitflyer.com/v1/me/sendchildorder 400 Bad Request {"status":-100,"error_message":"Invalid product","data":null}
60秒待機してやり直します


KeyboardInterrupt: ignored